## Como criar uma tool?

### Função de exemplo

In [2]:
import yfinance as yf

def obter_cotacao_atual(acao: str):
    try:
        ticker = yf.Ticker(acao)
        dados = ticker.history(period="1d")  

        if not dados.empty:
            ultima_linha = dados.iloc[-1]
            return {
                "acao": acao,
                "preco_atual": ultima_linha["Close"],
                "ultima_atualizacao": ultima_linha.name.strftime("%Y-%m-%d %H:%M:%S"),
            }
        else:
            return {"erro": "Nenhum dado encontrado para a ação fornecida."}
    except Exception as e:
        return {"erro": str(e)}

cotacao = obter_cotacao_atual("AAPL")
print(cotacao)

{'acao': 'AAPL', 'preco_atual': np.float64(239.6999969482422), 'ultima_atualizacao': '2025-02-28 00:00:00'}


### Tool de CrewAI

In [ ]:
from crewai.tools import BaseTool
import yfinance as yf
import json


class RetornaCotacaoAtual(BaseTool):
    name: str = 'Retorna cotação atual'
    description: str = 'Obtém a cotação mais recente de uma ação através da API do Yahoo Finance'

    def _run(self, acao: str) -> str:
        try:
            ticker = yf.Ticker(acao)
            dados = ticker.history(period="1d")  

            if not dados.empty:
                ultima_linha = dados.iloc[-1]
                return json.dumps({
                    "acao": acao,
                    "preco_atual": ultima_linha["Close"],
                    "ultima_atualizacao": ultima_linha.name.strftime("%Y-%m-%d %H:%M:%S"),
                })
            else:
                return json.dumps({"erro": "Nenhum dado encontrado para a ação fornecida."})
        except Exception as e:
            return json.dumps({"erro": str(e)})

'{"acao": "AAPL", "preco_atual": 240.85000610351562, "ultima_atualizacao": "2025-02-28 00:00:00"}'

### Versão profissional

In [ ]:
from crewai.tools import BaseTool
import yfinance as yf
import json
from typing import Type

from pydantic import BaseModel, Field

class RetornaCotacaoAtualInput(BaseModel):
    '''Argumentos da tool RetornaCotacaoAtual'''
    acao: str = Field(description='Ticker da ação conforme Yahoo Finance (ex: AAPL para Apple, PETR4.SA para Petrobras, etc)')


class RetornaCotacaoAtual(BaseTool):
    name: str = 'Retorna cotação atual'
    description: str = 'Obtém a cotação mais recente de uma ação através da API do Yahoo Finance'
    args_schema: Type[BaseModel] = RetornaCotacaoAtualInput

    def _run(self, acao: str) -> str:
        try:
            ticker = yf.Ticker(acao)
            dados = ticker.history(period="1d")  

            if not dados.empty:
                ultima_linha = dados.iloc[-1]
                return json.dumps({
                    "acao": acao,
                    "preco_atual": ultima_linha["Close"],
                    "ultima_atualizacao": ultima_linha.name.strftime("%Y-%m-%d %H:%M:%S"),
                })
            else:
                return json.dumps({"erro": "Nenhum dado encontrado para a ação fornecida."})
        except Exception as e:
            return json.dumps({"erro": str(e)})